In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler

In [156]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [410]:
df_original = pd.read_csv('data_fusionada.csv')

## Filtrar usuarios aptos para el proceso de recomendación **item-item**

### Trabajando columna **'recommend'**

In [389]:
# Cargar el DataFrame
columnas = ['item_id', 'genres', 'playtime_forever', 'recommend']
df_item = pd.read_csv('data_fusionada.csv', usecols=columnas)

df_item.head()

,item_id,genres,playtime_forever,recommend
0,282010,Action,466.0,True
1,70,Action,1395.0,True
2,70,Action,590.0,True
3,70,Action,5599.0,True
4,70,Action,64.0,True


In [390]:
# Vemos las dimensiones del DF
df_item.shape

(40015, 4)

In [391]:
# Agrupamos por interacciones 'item_id' y contamos las 'recommed' para cada juego
frecuencia_juegos = df_item[df_item['recommend'] == True]['item_id'].value_counts().reset_index()
frecuencia_juegos.columns = ['item_id', 'frecuencia_rec']
frecuencia_juegos

,item_id,frecuencia_rec
0,730,3119
1,4000,1491
2,304930,849
3,218620,775
4,550,639
...,...,...
2459,457580,1
2460,446120,1
2461,454320,1
2462,457520,1


In [392]:
# Unimos el dataframe principal con el DF 'frecuencia de juegos'
df_merge1 = pd.merge(df_item, frecuencia_juegos, on='item_id', how='left')
df_merge1['frecuencia_rec'] = df_merge1['frecuencia_rec'].fillna(0)
df_merge1

,item_id,genres,playtime_forever,recommend,frecuencia_rec
0,282010,Action,466.0,True,1.0
1,70,Action,1395.0,True,52.0
2,70,Action,590.0,True,52.0
3,70,Action,5599.0,True,52.0
4,70,Action,64.0,True,52.0
...,...,...,...,...,...
40010,80,Action,2416.0,False,8.0
40011,80,Action,15.0,False,8.0
40012,80,Action,442.0,True,8.0
40013,80,Action,72.0,True,8.0


In [393]:
# Convertir la columna 'frecuencia_rec' a tipo de dato entero.
df_merge1['frecuencia_rec'] = df_merge1['frecuencia_rec'].astype(int)
df_merge1.head()

,item_id,genres,playtime_forever,recommend,frecuencia_rec
0,282010,Action,466.0,True,1
1,70,Action,1395.0,True,52
2,70,Action,590.0,True,52
3,70,Action,5599.0,True,52
4,70,Action,64.0,True,52


In [395]:
df_merge1['recommend'].value_counts()

recommend
True     35843
False     4172
Name: count, dtype: int64

Filtrar datos relevantes en df_merge1 basándose en la frecuencia de recomendación

In [396]:
filtro_rec = df_merge1[df_merge1['frecuencia_rec'] >= 5]
filtro_rec

,item_id,genres,playtime_forever,recommend,frecuencia_rec
1,70,Action,1395.0,True,52
2,70,Action,590.0,True,52
3,70,Action,5599.0,True,52
4,70,Action,64.0,True,52
5,70,Action,3235.0,True,52
...,...,...,...,...,...
40010,80,Action,2416.0,False,8
40011,80,Action,15.0,False,8
40012,80,Action,442.0,True,8
40013,80,Action,72.0,True,8


In [397]:
# Verificamos la cantidad de 'item_id' tenemos para trabajar
filtro_rec['item_id'].nunique()

802

In [398]:
# Seleccionar columnas relevantes en df_f_rec para análisis posterior
columnas = ['item_id', 'genres', 'frecuencia_rec']
df_f_rec = filtro_rec[columnas]

In [293]:
# Eliminar duplicados manteniendo la primera ocurrencia
df_f_rec_agrupado = df_f_rec.drop_duplicates(subset=['item_id'])
df_f_rec_agrupado

,item_id,genres,frecuencia_rec
1,70,Action,52
54,2400,Action,21
77,1520,Indie,5
86,4000,Indie,1491
1619,2990,Racing,6
...,...,...,...
39935,60,Action,8
39943,10,Action,44
39989,130,Action,5
39998,13230,Action,6


In [399]:
df_f_rec_agrupado.shape

(802, 3)

Terminamos el trabajo de la columna **recommend**, consiguiendo disminuir a 802 juegos **'item_id'**, este filtro nos facilitará el proceso de ML

### Trabajo columna **'playtime_forever'**

In [400]:
# Cargar el DataFrame
columnas = ['item_id', 'genres', 'playtime_forever', 'recommend']
df_item2 = pd.read_csv('data_fusionada.csv', usecols=columnas)
df_item2.head()

,item_id,genres,playtime_forever,recommend
0,282010,Action,466.0,True
1,70,Action,1395.0,True
2,70,Action,590.0,True
3,70,Action,5599.0,True
4,70,Action,64.0,True


In [401]:
# Sumar el tiempo de juego para cada juego (item_id)
playtime = df_item2.groupby('item_id')['playtime_forever'].sum().reset_index()
playtime.columns = ['item_id', 'total_playtime']
playtime


,item_id,total_playtime
0,10,747975.0
1,20,72492.0
2,30,2392.0
3,40,145.0
4,50,1234.0
...,...,...
2717,521340,5.0
2718,521430,5.0
2719,521570,79.0
2720,521990,16.0


In [403]:
# Consultando la cantidad de juegos tenemos 'item_id'
playtime['item_id'].nunique()

2722

In [404]:
# Ordenar por la columna 'playtime_forever' de manera ascendente
playtime_ord = playtime.sort_values(by='total_playtime')
playtime_ord.head(10)

,item_id,total_playtime
2173,362280,1.0
401,45710,1.0
2143,357380,1.0
104,7760,1.0
1901,328830,5.0
520,96200,5.0
2458,407600,5.0
1722,307990,5.0
1840,321610,5.0
2232,369060,5.0


In [406]:
# Al verificar que nuestra columna total_playtime tiene datos booleanos, los cambiamos a enteros, para optimizar los futuros procesos.
playtime_ord['total_playtime'] = playtime_ord['total_playtime'].astype(int)

In [407]:
#Filtrar juegos con tiempo de juego superior a 2000 minutos
filtro_time = playtime_ord[playtime_ord['total_playtime'] >= 2000]
filtro_time

,item_id,total_playtime
1267,262490,2000
1537,289600,2001
2263,374040,2027
176,12590,2028
2499,413420,2031
...,...,...
552,105600,10320781
892,230410,12321959
771,218620,12546425
68,4000,49464882


Luego de trabajar con **'playtime_forever**  hemos logrado identificar a los datos de juegos más optimos para los siguientes procesos

In [415]:
# Crear un DataFrame con las columnas necesarias
columnas = ['item_id', 'genres']
df_genres = df_original[columnas]

frecuencia_genres = df_genres.groupby(['item_id', 'genres']).size().reset_index(name='frecuencia_genres')

# Consolidar la información agrupando por item_id
df_genres_agrupado = frecuencia_genres.groupby('item_id', as_index=False).agg({'genres': ' | '.join, 'frecuencia_genres': 'sum'})
df_genres_agrupado

,item_id,genres,frecuencia_genres
0,10,Action,45
1,20,Action,15
2,30,Action,4
3,40,Action,1
4,50,Action,3
...,...,...,...
2717,521340,Casual,1
2718,521430,Indie,1
2719,521570,Casual,2
2720,521990,Action,1


In [412]:
# Verificamos la cantidad de juegos que entrarían a futuros proceso ML
df_genres_agrupado['item_id'].nunique()

2722

In [416]:
filtro_genres= df_genres_agrupado[df_genres_agrupado['frecuencia_genres'] >= 5]
filtro_genres

,item_id,genres,frecuencia_genres
0,10,Action,45
1,20,Action,15
5,60,Action,8
6,70,Action,53
7,80,Action,11
...,...,...,...
2611,444090,Action,13
2630,449140,Indie,5
2649,459820,Casual,5
2654,461560,Adventure,5


In [418]:
# Verificamos nuevamente como ha cambiado la cantidad de juegos
filtro_genres['item_id'].nunique()

880

Luego de trabajar con **'frecuencia_genres'**  hemos logrado identificar 880 juegos con mejor data, para nuestros siguientes proceso

## Guardar DF que servirá para realizar el proceso de ML recomendación 'item'-'item'

In [427]:
# Unir los DataFrames
df_final = pd.merge(df_f_rec_agrupado, filtro_time, on='item_id')
df_final = pd.merge(df_final, filtro_genres, on='item_id')
df_final

,item_id,genres_x,frecuencia_rec,total_playtime,genres_y,frecuencia_genres
0,70,Action,52,56884,Action,53
1,2400,Action,21,10608,Action,22
2,1520,Indie,5,17155,Indie,7
3,4000,Indie,1491,49464882,Indie,1533
4,2990,Racing,6,5933,Racing,6
...,...,...,...,...,...,...
717,1510,Indie,5,5666,Indie,5
718,20,Action,10,72492,Action,15
719,10,Action,44,747975,Action,45
720,13230,Action,6,6616,Action,6


In [428]:
# Supongamos que quieres renombrar las columnas 'genres_x' y 'genres_y'
df_final = df_final.rename(columns={'frecuencia_genres': 'genres', 'frecuencia_rec': 'recommend','total_playtime':'playtime_forever'})
df_final

,item_id,genres_x,recommend,playtime_forever,genres_y,genres
0,70,Action,52,56884,Action,53
1,2400,Action,21,10608,Action,22
2,1520,Indie,5,17155,Indie,7
3,4000,Indie,1491,49464882,Indie,1533
4,2990,Racing,6,5933,Racing,6
...,...,...,...,...,...,...
717,1510,Indie,5,5666,Indie,5
718,20,Action,10,72492,Action,15
719,10,Action,44,747975,Action,45
720,13230,Action,6,6616,Action,6


In [429]:
columnas = ['item_id', 'genres', 'recommend', 'playtime_forever']
df_final = df_final[columnas]
df_final

,item_id,genres,recommend,playtime_forever
0,70,53,52,56884
1,2400,22,21,10608
2,1520,7,5,17155
3,4000,1533,1491,49464882
4,2990,6,6,5933
...,...,...,...,...
717,1510,5,5,5666
718,20,15,10,72492
719,10,45,44,747975
720,13230,6,6,6616


In [430]:
# Mergeamos con la data original, usando 'item_id'. Gracias al trabajo realizado anteriormente, vamos a obtener filas de nuestros juegos mejor data.
df_resultado = pd.merge(df_final, df_original[['item_id', 'app_name']], on='item_id', how='inner')
df_resultado

,item_id,genres,recommend,playtime_forever,app_name
0,70,53,52,56884,Half-Life
1,70,53,52,56884,Half-Life
2,70,53,52,56884,Half-Life
3,70,53,52,56884,Half-Life
4,70,53,52,56884,Half-Life
...,...,...,...,...,...
35738,80,11,8,37715,Counter-Strike: Condition Zero
35739,80,11,8,37715,Counter-Strike: Condition Zero
35740,80,11,8,37715,Counter-Strike: Condition Zero
35741,80,11,8,37715,Counter-Strike: Condition Zero


In [431]:
# Eliminamos duplicados y nulos
df_resultado = df_resultado.drop_duplicates()
df_resultado = df_resultado.dropna()
df_resultado

,item_id,genres,recommend,playtime_forever,app_name
0,70,53,52,56884,Half-Life
53,2400,22,21,10608,The Ship: Murder Party
75,1520,7,5,17155,DEFCON
82,4000,1533,1491,49464882,Garry's Mod
1615,2990,6,6,5933,FlatOut 2™
...,...,...,...,...,...
35661,1510,5,5,5666,Uplink
35666,20,15,10,72492,Team Fortress Classic
35681,10,45,44,747975,Counter-Strike
35726,13230,6,6,6616,Unreal Tournament 2004: Editor's Choice Edition


In [432]:
# Ordenamos las columnas, para facilitar el futuro proceso de ML
columnas = ['item_id','app_name','recommend','playtime_forever','genres']
df_ok_item_item = df_resultado[columnas]
df_ok_item_item

,item_id,app_name,recommend,playtime_forever,genres
0,70,Half-Life,52,56884,53
53,2400,The Ship: Murder Party,21,10608,22
75,1520,DEFCON,5,17155,7
82,4000,Garry's Mod,1491,49464882,1533
1615,2990,FlatOut 2™,6,5933,6
...,...,...,...,...,...
35661,1510,Uplink,5,5666,5
35666,20,Team Fortress Classic,10,72492,15
35681,10,Counter-Strike,44,747975,45
35726,13230,Unreal Tournament 2004: Editor's Choice Edition,6,6616,6


In [433]:
# Guardamos nuestro df para el proceso de ML recomendaciones
df_ok_item_item.to_csv('dataML_item_item.csv', index=False)

## Filtrar usuarios aptos para el proceso de recomendación **user-item**


In [64]:
# Cargar el DataFrame
columnas = ['user_id', 'item_id', 'recommend', 'app_name', 'Sentiment_analysis', 'playtime_forever','genres','developer']
df = pd.read_csv('data_fusionada.csv', usecols=columnas)

In [65]:
df.head()

,app_name,item_id,developer,genres,playtime_forever,user_id,recommend,Sentiment_analysis
0,Carmageddon Max Pack,282010,Stainless Games Ltd,Action,466.0,InstigatorAU,True,1.0
1,Half-Life,70,Valve,Action,1395.0,EizanAratoFujimaki,True,2.0
2,Half-Life,70,Valve,Action,590.0,GamerFag,True,0.0
3,Half-Life,70,Valve,Action,5599.0,76561198020928326,True,2.0
4,Half-Life,70,Valve,Action,64.0,Bluegills,True,2.0


In [66]:
# Creamos una columna que nos facilita obtener la información sobre la cantidad de interacciones 
df['recommend1'] = 1
df

,app_name,item_id,developer,genres,playtime_forever,user_id,recommend,Sentiment_analysis,recommend1
0,Carmageddon Max Pack,282010,Stainless Games Ltd,Action,466.0,InstigatorAU,True,1.0,1
1,Half-Life,70,Valve,Action,1395.0,EizanAratoFujimaki,True,2.0,1
2,Half-Life,70,Valve,Action,590.0,GamerFag,True,0.0,1
3,Half-Life,70,Valve,Action,5599.0,76561198020928326,True,2.0,1
4,Half-Life,70,Valve,Action,64.0,Bluegills,True,2.0,1
...,...,...,...,...,...,...,...,...,...
40010,Counter-Strike: Condition Zero,80,Valve,Action,2416.0,76561198015050660,False,1.0,1
40011,Counter-Strike: Condition Zero,80,Valve,Action,15.0,76561198023508728,False,1.0,1
40012,Counter-Strike: Condition Zero,80,Valve,Action,442.0,green290,True,1.0,1
40013,Counter-Strike: Condition Zero,80,Valve,Action,72.0,174gamecuman700kngkakak,True,1.0,1


In [67]:
resultados = df.groupby('user_id').agg({
    'playtime_forever': 'sum',
    'recommend1': 'sum',
    'developer': lambda x: sum(len(str(y).strip()) for y in x if isinstance(y, str)),
    'genres': lambda x: sum(len(str(y).strip()) for y in x if isinstance(y, str))
}).reset_index()

# Renombrar las columnas
resultados = resultados.rename(columns={'developer': 'developer_sum', 'genres': 'genres_sum'})
resultados

,user_id,playtime_forever,recommend1,developer_sum,genres_sum
0,--000--,2949.0,1,20,6
1,--ace--,1269.0,1,33,6
2,--ionex--,36320.0,2,13,12
3,-2SV-vuLB-Kg,31337.0,3,74,20
4,-Azsael-,11889.0,1,22,5
...,...,...,...,...,...
18951,zuzuga2003,72732.0,4,63,23
18952,zv_odd,6452.0,8,91,50
18953,zvanik,38959.0,3,36,18
18954,zynxgameth,3846.0,1,12,6


In [68]:
# Ordenar por la columna 'playtime_forever' de manera ascendente
df_sorted = resultados.sort_values(by='genres_sum')

# Mostrar el DataFrame ordenado
df_sorted.head()

,user_id,playtime_forever,recommend1,developer_sum,genres_sum
15765,joshfeb06,304.0,1,19,3
18186,thecoolone3,6997.0,1,21,3
1823,76561198041749839,12222.0,1,11,3
18000,sugarino,2172.0,1,17,3
651,76561197995108212,679.0,1,21,3


In [70]:
df_filtro_time = resultados[resultados['playtime_forever'] >= 2000]
df_filtro_time

,user_id,playtime_forever,recommend1,developer_sum,genres_sum
0,--000--,2949.0,1,20,6
2,--ionex--,36320.0,2,13,12
3,-2SV-vuLB-Kg,31337.0,3,74,20
4,-Azsael-,11889.0,1,22,5
5,-Beave-,2864.0,1,5,6
...,...,...,...,...,...
18951,zuzuga2003,72732.0,4,63,23
18952,zv_odd,6452.0,8,91,50
18953,zvanik,38959.0,3,36,18
18954,zynxgameth,3846.0,1,12,6


In [72]:
df_filtro_rec = df_filtro_time[df_filtro_time['recommend1'] >= 5]
df_filtro_rec

,user_id,playtime_forever,recommend1,developer_sum,genres_sum
9,-SEVEN-,187309.0,9,155,74
42,091263,2340.0,9,168,57
51,1011001,45293.0,8,111,51
54,10Degree,38685.0,5,61,29
57,111222333444555666888,42999.0,5,121,30
...,...,...,...,...,...
18921,zerzang,4509.0,5,80,29
18935,zombi_anon,11404.0,5,93,30
18942,zomgieee,39702.0,7,96,41
18952,zv_odd,6452.0,8,91,50


In [166]:
# Supongamos que 'user_id' es la columna que contiene los identificadores de usuario en ambos DataFrames
usuarios_filtrados = df_main[df_main['user_id'].isin(df_filtro_rec['user_id'])]

# Ahora, 'usuarios_filtrados' contiene solo las filas de 'df_main' que corresponden a los 4981 usuarios en 'df_nuevo'
usuarios_filtrados

,app_name,item_id,playtime_forever,user_id,recommend,Sentiment_analysis
2,Half-Life,70,590.0,GamerFag,True,0.0
4,Half-Life,70,64.0,Bluegills,True,2.0
5,Half-Life,70,3235.0,76561198071955492,True,2.0
6,Half-Life,70,1301.0,digfernandes,True,0.0
7,Half-Life,70,21.0,76561197970812298,True,2.0
...,...,...,...,...,...,...
40009,Counter-Strike: Condition Zero,80,3721.0,KILLERamateur,True,1.0
40010,Counter-Strike: Condition Zero,80,2416.0,76561198015050660,False,1.0
40012,Counter-Strike: Condition Zero,80,442.0,green290,True,1.0
40013,Counter-Strike: Condition Zero,80,72.0,174gamecuman700kngkakak,True,1.0


In [173]:
df_main

,app_name,item_id,playtime_forever,user_id,recommend,Sentiment_analysis
0,Carmageddon Max Pack,282010,466.0,InstigatorAU,True,1.0
1,Half-Life,70,1395.0,EizanAratoFujimaki,True,2.0
2,Half-Life,70,590.0,GamerFag,True,0.0
3,Half-Life,70,5599.0,76561198020928326,True,2.0
4,Half-Life,70,64.0,Bluegills,True,2.0
...,...,...,...,...,...,...
40010,Counter-Strike: Condition Zero,80,2416.0,76561198015050660,False,1.0
40011,Counter-Strike: Condition Zero,80,15.0,76561198023508728,False,1.0
40012,Counter-Strike: Condition Zero,80,442.0,green290,True,1.0
40013,Counter-Strike: Condition Zero,80,72.0,174gamecuman700kngkakak,True,1.0


In [172]:
usuarios_filtrados['Sentiment_analysis'].unique()

array([0, 2, 1])

In [167]:
usuarios_filtrados['Sentiment_analysis'] = usuarios_filtrados['Sentiment_analysis'].astype(int)

C:\Users\Isaac\AppData\Local\Temp\ipykernel_12948\696766697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios_filtrados['Sentiment_analysis'] = usuarios_filtrados['Sentiment_analysis'].astype(int)


In [168]:
# Función para visualizar la calidad de nuestros datos

def resumen_tipos_de_datos(df):
    
    resumen = []

    for col in df.columns:

        tipos_presentes = df[col].apply(lambda x: type(x).__name__).unique()
        cantidad_nulos = df[col].isnull().sum()
        resumen.append({
            'Columna': col,
            'Tipos de Dato': tipos_presentes.tolist(),
            'Cantidad de Nulos': cantidad_nulos
        })   
    return pd.DataFrame(resumen)

In [169]:
resumen_tipos_de_datos(usuarios_filtrados)

,Columna,Tipos de Dato,Cantidad de Nulos
0,app_name,[str],0
1,item_id,[int],0
2,playtime_forever,[float],0
3,user_id,[str],0
4,recommend,[bool],0
5,Sentiment_analysis,[int],0


In [ ]:
# Guardamos nuestro df para el proceso de ML recomendaciones
usuarios_filtrados.to_csv('dataML_user_item.csv', index=False)

In [387]:
usuarios_filtrados

,app_name,item_id,playtime_forever,user_id,recommend,Sentiment_analysis
2,Half-Life,70,590.0,GamerFag,True,0
4,Half-Life,70,64.0,Bluegills,True,2
5,Half-Life,70,3235.0,76561198071955492,True,2
6,Half-Life,70,1301.0,digfernandes,True,0
7,Half-Life,70,21.0,76561197970812298,True,2
...,...,...,...,...,...,...
40009,Counter-Strike: Condition Zero,80,3721.0,KILLERamateur,True,1
40010,Counter-Strike: Condition Zero,80,2416.0,76561198015050660,False,1
40012,Counter-Strike: Condition Zero,80,442.0,green290,True,1
40013,Counter-Strike: Condition Zero,80,72.0,174gamecuman700kngkakak,True,1
